In [ ]:
# Structure_offre_TV.csv et structure_offre_TV.csv proviennent de la section "O&C" de Audience de la télévision.xlsx

In [ ]:
# Pour le reste, veuillez importer lesfrancaisetlinformationpropre.csv s'il vous plait car tout les autres fichiers créés s'appuient sur celui-ci

In [ ]:
# Création de "age_format3_frequencies.csv" qui calcule le nombre de chaque type de réponse de la colonne FORMAT3_LR3_FORMAT3_3_R de lesfrancaisetlinformationpropre.csv, pour chaque tranche d'age

In [1]:
import pandas as pd

def calculate_age_format_frequencies():
    # Lecture du fichier CSV avec l'encodage cp1252
    df = pd.read_csv('lesfrancaisetlinformationpropre.csv', encoding='cp1252')

    # Calcul des fréquences croisées entre l'âge et le format
    frequencies = pd.crosstab(df['RS2C_RECODE_AG_R'],
                            df['FORMAT3_LR3_FORMAT3_3_R'],
                            margins=False)

    # Convertir en format long (une ligne par combinaison)
    frequencies_long = frequencies.reset_index().melt(
        id_vars=['RS2C_RECODE_AG_R'],
        var_name='FORMAT3_LR3_FORMAT3_3_R',
        value_name='Frequency'
    )

    # Trier par tranche d'âge
    frequencies_long = frequencies_long.sort_values(['RS2C_RECODE_AG_R', 'FORMAT3_LR3_FORMAT3_3_R'])

    # Sauvegarder le résultat
    frequencies_long.to_csv('age_format3_frequencies.csv', index=False, encoding='utf-8')

    return "Analyse terminée. Les résultats ont été sauvegardés dans age_format3_frequencies.csv"

# Exécuter l'analyse
if __name__ == "__main__":
    result = calculate_age_format_frequencies()
    print(result)

<ipython-input-1-79c15f39a325>:5: DtypeWarning: Columns (16,20,104,105,111,112,113,120,121,128,129,136,137,145,152,153,234,264,265,279,280,281,282,296,305,316,317,318,330,342,343,344) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('lesfrancaisetlinformationpropre.csv', encoding='cp1252')


Analyse terminée. Les résultats ont été sauvegardés dans age_format3_frequencies.csv


In [ ]:
# Création de "Feuille1_converted.csv" qui correspond aux fréquences additionnées des gens ayant répondu "Au moins une fois par jour" et "Plusieurs fois par jour" dans la colonne NEWS1_R_2 en fonction des différentes tranches d'age

In [3]:
import pandas as pd

def calculate_daily_media_frequencies():
    # Lecture du fichier CSV
    df = pd.read_csv('lesfrancaisetlinformationpropre.csv', encoding='latin1')

    # Liste des colonnes NEWS1_R pour chaque média
    media_columns = {
        'Radio/Podcasts': 'NEWS1_R_1',
        'Télévision': 'NEWS1_R_2',
        'Journal/Magazines': 'NEWS1_R_3',
        'Sites Internet': 'NEWS1_R_4',
        'Moteurs de Recherche': 'NEWS1_R_5',
        'Portails d\'actualité': 'NEWS1_R_6',
        'Réseaux sociaux': 'NEWS1_R_7',
        'Plateformes vidéo': 'NEWS1_R_8'
    }

    # Définir les réponses quotidiennes
    daily_responses = ['Plusieurs fois par jour', 'Au moins une fois par jour']

    # Créer un DataFrame pour stocker les résultats
    results = []

    # Calculer les pourcentages pour chaque tranche d'âge
    age_groups = df['RS2C_RECODE_AG_R'].unique()

    for age_group in sorted(age_groups, key=lambda x: int(x.split('-')[0]) if '-' in x else (0 if 'moins' in x.lower() else 100)):
        row = {'Unnamed: 0': age_group}
        age_group_total = len(df[df['RS2C_RECODE_AG_R'] == age_group])

        for media_name, column in media_columns.items():
            # Calculer le nombre d'utilisateurs quotidiens
            daily_users = len(df[
                (df['RS2C_RECODE_AG_R'] == age_group) &
                (df[column].isin(daily_responses))
            ])

            # Calculer le pourcentage
            percentage = (daily_users / age_group_total) * 100 if age_group_total > 0 else 0
            row[media_name] = round(percentage, 1)

        results.append(row)

    # Créer le DataFrame final
    result_df = pd.DataFrame(results)

    # Sauvegarder en CSV
    result_df.to_csv('Feuille1_converted.csv', index=False, encoding='utf-8')

    return "Feuille1_converted.csv a été créé avec succès"

# Exécuter la fonction
if __name__ == "__main__":
    print(calculate_daily_media_frequencies())

<ipython-input-3-9fb3ecb69297>:5: DtypeWarning: Columns (16,20,104,105,111,112,113,120,121,128,129,136,137,145,152,153,234,264,265,279,280,281,282,296,305,316,317,318,330,342,343,344) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('lesfrancaisetlinformationpropre.csv', encoding='latin1')


Feuille1_converted.csv a été créé avec succès


In [ ]:
# Création de social_media_frequencies.csv il prend en compte la colonne FORMAT3_LR3_FORMAT3_3_R, la colonne RSINFO_2_LR_R_2 et la colonne RSINFO_2_LR_R_4 de lesfrancaisetlinformationpropre.csv, et qui calcule la fréquence pour chaque cas de figure

In [12]:
import pandas as pd

def calculate_social_media_frequencies():
    # Lecture du fichier CSV source
    df = pd.read_csv('lesfrancaisetlinformationpropre.csv', encoding='latin1')

    # Sélectionner les colonnes pertinentes
    columns_of_interest = [
        'RS2C_RECODE_AG_R',
        'FORMAT3_LR3_FORMAT3_3_R',
        'RSINFO_2_LR_R_2',
        'RSINFO_2_LR_R_4'
    ]

    # Créer un DataFrame avec uniquement les colonnes qui nous intéressent
    df_filtered = df[columns_of_interest].copy()

    # Calculer les fréquences absolues pour chaque combinaison unique
    frequencies = df_filtered.groupby(columns_of_interest).size().reset_index(name='count')

    # Calculer les totaux par tranche d'âge et format
    def calculate_relative_frequencies(group):
        total = group['count'].sum()
        group['Frequency'] = group['count'] / total if total > 0 else 0
        return group

    # Grouper par tranche d'âge et format, puis calculer les fréquences relatives
    frequencies = frequencies.groupby(['RS2C_RECODE_AG_R', 'FORMAT3_LR3_FORMAT3_3_R'], group_keys=False).apply(calculate_relative_frequencies)

    # Sélectionner et trier les colonnes finales
    frequencies = frequencies[columns_of_interest + ['Frequency']].sort_values([
        'RS2C_RECODE_AG_R',
        'FORMAT3_LR3_FORMAT3_3_R',
        'RSINFO_2_LR_R_2',
        'RSINFO_2_LR_R_4'
    ])

    # Sauvegarder le résultat en CSV
    frequencies.to_csv('social_media_frequencies.csv', index=False, encoding='utf-8')

    return "social_media_frequencies.csv a été créé avec succès"

# Exécuter la fonction
if __name__ == "__main__":
    print(calculate_social_media_frequencies())

social_media_frequencies.csv a été créé avec succès


<ipython-input-12-30bb7197888d>:5: DtypeWarning: Columns (16,20,104,105,111,112,113,120,121,128,129,136,137,145,152,153,234,264,265,279,280,281,282,296,305,316,317,318,330,342,343,344) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('lesfrancaisetlinformationpropre.csv', encoding='latin1')
<ipython-input-12-30bb7197888d>:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  frequencies = frequencies.groupby(['RS2C_RECODE_AG_R', 'FORMAT3_LR3_FORMAT3_3_R'], group_keys=False).apply(calculate_relative_frequencies)


In [ ]:
# Création de filtered_age_group_and_main_info_mode.csv,  c'est juste la colonne NEWS1BIS_R de lesfrancaisetlinformationpropre.csv

In [14]:
import pandas as pd

def create_filtered_age_info_mode():
    # Lecture du fichier CSV source
    df = pd.read_csv('lesfrancaisetlinformationpropre.csv', encoding='latin1')

    # Sélectionner uniquement les colonnes qui nous intéressent
    columns_of_interest = [
        'RS2C_RECODE_AG_R',
        'NEWS1BIS_R'
    ]

    # Créer un nouveau DataFrame avec ces colonnes
    df_filtered = df[columns_of_interest].copy()

    # Sauvegarder le résultat en CSV
    df_filtered.to_csv('filtered_age_group_and_main_info_mode.csv', index=False, encoding='utf-8')

    return "filtered_age_group_and_main_info_mode.csv a été créé avec succès"

# Exécuter la fonction
if __name__ == "__main__":
    print(create_filtered_age_info_mode())

filtered_age_group_and_main_info_mode.csv a été créé avec succès


<ipython-input-14-941cdbb3d8fb>:5: DtypeWarning: Columns (16,20,104,105,111,112,113,120,121,128,129,136,137,145,152,153,234,264,265,279,280,281,282,296,305,316,317,318,330,342,343,344) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('lesfrancaisetlinformationpropre.csv', encoding='latin1')


In [ ]:
# Création de age_group_main_info_mode_frequencies.csv qui calcule la fréquence pour chaque tranche d'age, de réponses à la colonne NEWS1BIS_R de lesfrancaisetlinformationpropre.csv

In [28]:
import pandas as pd

def calculate_age_group_frequencies():
    # Lecture du fichier CSV source
    df = pd.read_csv('lesfrancaisetlinformationpropre.csv', encoding='latin1')

    # Fonction pour assigner le mode en utilisant des mots clés
    def assign_mode(x):
        if pd.isna(x):
            return None
        x = x.lower()
        if 'plateforme' in x and 'vidéo' in x:
            return 'Plateformes de vidéo en ligne (Youtube,Twitch, ...)'
        elif 'réseaux sociaux' in x:
            return 'Réseaux sociaux'
        elif 'journal ou' in x:
            return 'Journal ou Magazine '
        elif 'télévision' in x:
            return 'Télévision'
        elif 'moteur de recherche' in x:
            return 'Moteur de recherche'
        elif 'portail d\'actualité' in x:
            return 'Portail d\'actualité'
        elif 'média disponible uniquement sur internet' in x or 'mediapart' in x:
            return 'Média disponible uniquement sur Internet'
        elif 'radio' in x:
            return 'Radio, podcast'
        return None

    # Appliquer la fonction de mapping
    df['Mode'] = df['NEWS1BIS_R'].apply(assign_mode)

    # Créer un DataFrame pour stocker les résultats
    age_groups = ['15-17 ans', '18-24 ans', '25-34 ans', '35-44 ans',
                 '45-59 ans', '60-69 ans', '70 ans et +']

    result_df = pd.DataFrame(index=age_groups)

    # Liste des modes dans l'ordre souhaité
    desired_columns = [
        'Plateformes de vidéo en ligne (Youtube,Twitch, ...)',
        'Réseaux sociaux',
        'Journal ou Magazine ',
        'Télévision',
        'Moteur de recherche',
        'Portail d\'actualité',
        'Média disponible uniquement sur Internet',
        'Radio, podcast'
    ]

    # Pour chaque mode d'information
    for mode in desired_columns:
        # Calculer la fréquence pour chaque tranche d'âge
        frequencies = []
        for age in age_groups:
            age_total = len(df[df['RS2C_RECODE_AG_R'] == age])
            mode_count = len(df[(df['RS2C_RECODE_AG_R'] == age) & (df['Mode'] == mode)])
            freq = mode_count / age_total if age_total > 0 else 0
            frequencies.append(freq)

        result_df[mode] = frequencies

    # Ajouter la colonne Age_Group
    result_df.index.name = 'Age_Group'
    result_df = result_df.reset_index()

    # Sauvegarder les résultats
    result_df.to_csv('age_group_main_info_mode_frequencies.csv', index=False, encoding='utf-8')

    # Debug: afficher quelques valeurs pour vérification
    print("\nPremières lignes du résultat:")
    print(result_df.head())

    return "age_group_main_info_mode_frequencies.csv a été créé avec succès"

# Exécuter la fonction
if __name__ == "__main__":
    print(calculate_age_group_frequencies())

<ipython-input-28-adcebcfbbe64>:5: DtypeWarning: Columns (16,20,104,105,111,112,113,120,121,128,129,136,137,145,152,153,234,264,265,279,280,281,282,296,305,316,317,318,330,342,343,344) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('lesfrancaisetlinformationpropre.csv', encoding='latin1')



Premières lignes du résultat:
   Age_Group  Plateformes de vidéo en ligne (Youtube,Twitch, ...)  \
0  15-17 ans                                           0.095238     
1  18-24 ans                                           0.095406     
2  25-34 ans                                           0.074879     
3  35-44 ans                                           0.031189     
4  45-59 ans                                           0.018888     

   Réseaux sociaux  Journal ou Magazine   Télévision  Moteur de recherche  \
0         0.333333              0.028571    0.257143             0.085714   
1         0.342756              0.038869    0.183746             0.144876   
2         0.200483              0.065217    0.236715             0.181159   
3         0.181287              0.064327    0.278752             0.161793   
4         0.108080              0.075551    0.402938             0.104932   

   Portail d'actualité  Média disponible uniquement sur Internet  \
0             0.104762 